## Exploring vegetation and Burned area datasets using Magics and xarray

This notebooks helps us analyse AGB data and burned area data, by visualising the products together. 
Both datasets have monthly time steps for period January to December 2017 and cover global area.

In [1]:
from __future__ import print_function
import xarray as xr
import pandas as pd
import Magics.macro as magics
import ipywidgets as widgets
from datetime import date,timedelta,datetime

from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from IPython.display import clear_output, display

#### Loading AGB data and burned area data from CDS
Data downloaded from Climate Data Store (CDS) matches the time frame of vegetation (AGB) data (2017), so they can be plotted together for start.

In [2]:
abg_avitabile_vod5th = xr.open_mfdataset('data/abg_avitabile_vod5th_2017*.nc',combine='nested',concat_dim='time')
abg_avitabile_vod95th = xr.open_mfdataset('data/abg_avitabile_vod95th_2017*.nc',combine='nested',concat_dim='time')
abg_avitabile_vodmean = xr.open_mfdataset('data/abg_avitabile_vodmean_2017*.nc',combine='nested',concat_dim='time')
ba_ds  = xr.open_mfdataset('data/2017*-ESACCI-L4_FIRE-BA-MODIS-fv5.1cds.nc',combine='nested',concat_dim='time')

veg_ds = xr.merge([abg_avitabile_vod5th, abg_avitabile_vod95th, abg_avitabile_vodmean])

However, if we have a look at the time dimension of Burned area dataset, we can see that there are some inconsistencies with the dates. Somewhere the date is first of month and elsewhere it is last day of the previous month. 

In [3]:
ba_ds.time

<xarray.DataArray 'time' (time: 12)>
array(['2017-01-01T00:00:00.000000000', '2017-02-01T00:00:00.000000000',
       '2017-03-01T00:00:00.000000000', '2017-03-31T00:00:00.000000000',
       '2017-04-30T00:00:00.000000000', '2017-05-31T00:00:00.000000000',
       '2017-06-30T00:00:00.000000000', '2017-07-31T00:00:00.000000000',
       '2017-08-31T00:00:00.000000000', '2017-09-30T00:00:00.000000000',
       '2017-11-01T00:00:00.000000000', '2017-12-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2017-01-01 2017-02-01 ... 2017-12-01
Attributes:
    standard_name:  time
    long_name:      time
    bounds:         time_bounds
    axis:           T

For the purpose of matching time dimensions of both datasets, we create a new pandas series with monthly time stamps from January 2010 to December 2017 and replace the times in ba_ds dataset.

In [4]:
time = pd.date_range('2017-01-01', freq='MS', periods=12)
ba_ds['time'] = time

Now, we can check that the time dimension is the same for both datasets.

In [5]:
ba_ds.time

<xarray.DataArray 'time' (time: 12)>
array(['2017-01-01T00:00:00.000000000', '2017-02-01T00:00:00.000000000',
       '2017-03-01T00:00:00.000000000', '2017-04-01T00:00:00.000000000',
       '2017-05-01T00:00:00.000000000', '2017-06-01T00:00:00.000000000',
       '2017-07-01T00:00:00.000000000', '2017-08-01T00:00:00.000000000',
       '2017-09-01T00:00:00.000000000', '2017-10-01T00:00:00.000000000',
       '2017-11-01T00:00:00.000000000', '2017-12-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2017-01-01 2017-02-01 ... 2017-12-01

In [6]:
veg_ds.time

<xarray.DataArray 'time' (time: 12)>
array(['2017-01-01T00:00:00.000000000', '2017-02-01T00:00:00.000000000',
       '2017-03-01T00:00:00.000000000', '2017-04-01T00:00:00.000000000',
       '2017-05-01T00:00:00.000000000', '2017-06-01T00:00:00.000000000',
       '2017-07-01T00:00:00.000000000', '2017-08-01T00:00:00.000000000',
       '2017-09-01T00:00:00.000000000', '2017-10-01T00:00:00.000000000',
       '2017-11-01T00:00:00.000000000', '2017-12-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2017-01-01 2017-02-01 ... 2017-12-01

Next cell is the magics plotting function. It takes xarray variable, the date and area from drop down menu in the cell below and does the plotting.

In [7]:
def vegetation_time(ba_time='2017-03-01',time_lag=0):
    '''Calculates the time one or two months before the time of Burned Area data
       input  : time for burned area, time lag for the vegetation data
       output : time for the vegetation data
    '''
    
    new_month = ba_time.month - time_lag                    # getting the MONTH for vegetation data
    
    # if ba_time.month was 1 or 2 new_month can be negative.. 
    # make it 11 or 12 in the previous year
    if new_month >0:
        new_time = date(ba_time.year, new_month, 1 )
    else:
        new_time = date(ba_time.year-1,12+new_month, 1)

    return new_time    

def plot(parameter='abg_avitabile_vod95th', 
         parameter_ba='burned_area', 
         vegetation_class = 1,
         time1='2017-03-01', 
         time_lag = 0, 
         area = 'europe'):
    
    #Setting the geographical area
    projection = magics.mmap(
        subpage_map_library_area = "on",
        subpage_map_area_name    = area,
        page_id_line    = 'off'
    )

    #Setting the background
    background = magics.mcoast(    
    map_coastline_sea_shade_colour  = 'sky',
    map_coastline_land_shade_colour = 'RGB(0.29,0.29,0.29)',
    map_grid                        = 'off',
    map_rivers                      = 'off',
    map_coastline_land_shade        = 'on',
    map_coastline_sea_shade         = 'on',
    map_label                       = 'off',
    map_coastline_colour            = 'RGB(0.49,0.49,0.49)')
    
    #Setting the foreground
    foreground = magics.mcoast(    
    map_grid                        = 'on',
    map_rivers                      = 'on',
    map_coastline_land_shade        = 'off',
    map_coastline_sea_shade         = 'off',
    map_label                       = 'off',
    map_coastline_colour            = 'RGB(0.49,0.49,0.49)')

    #Loading vegetation data 
    ba_time = datetime.strptime(time1, '%Y-%m-%d').date()   # time for burned area data
    veg_time = vegetation_time(ba_time,time_lag)            # time for vegetataion data
    print(veg_time)
    try:
        veg_one  = veg_ds.sel(time=veg_time)
        veg_data = magics.mxarray(xarray_dataset = veg_one, 
                                  xarray_variable_name = parameter)
    except:
        print('Sorry, check that your data is in range Jan 2017 - Dec 2017') #no vegetation data for Nov/Dec 2009

    

    #Loading burned area data
    ba_one  = ba_ds.sel(time=time1)
    if parameter_ba == 'burned_area_in_vegetation_class':
        ba_one = ba_one.isel(vegetation_class=vegetation_class)
    
    ba_long_name = ba_ds[parameter_ba].long_name                      # saving variable long name
    ba_one[parameter_ba] = ba_one[parameter_ba]/1000000.              # converting form m^2 to km^2
    ba_data = magics.mxarray(xarray_dataset = ba_one,                
                             xarray_variable_name = parameter_ba)
    
    

    veg_colour_list1 = ['#ffffe5', '#b3df97', '#6aba6f', 
                        '#3f9152', '#1b683a','#004529']
    veg_colour_list2 = ['rgb(247,252,240)','rgb(204,235,197)','rgb(168,221,181)',
                        'rgb(78,179,211)','rgb(43,140,190)','rgb(8,64,129)']
    
    #Defining the contour
    veg_contour = magics.mcont(
        legend                            = "on",
        contour                           = "off",
        contour_level_selection_type      = "level_list",
        contour_level_list                = [0.,100.,200.,300.,400.,500],
        contour_gradients_step_list       = [10,10,10,10,10],
        contour_label                     = "off",
        contour_shade                     = "on",
        contour_shade_colour_method       = "gradients",
        contour_gradients_technique       = "rgb",
        contour_shade_technique           = "cell_shading",
        contour_shade_cell_resolution     = 100,
        contour_gradients_colour_list     = veg_colour_list2,
        contour_gradients_waypoint_method = 'left' )

    ba_colour_list2 = ['rgba(255,255,224,0)', 'rgba(245,221,118,0.5)', '#eab55e', 
                      '#de8d45', '#d2612b', '#bd3411', 
                      '#91210b', '#661005', '#400000']    
    ba_colour_list1 = ['rgba(247,244,249,0)', 'rgb(231,225,239,0.5)', '#d4b9da', 
                       '#c994c7', '#df65b0', '#e7298a',
                       '#ce1256', '#980043', '#67001f']
    ba_colour_list3 = ['rgba(250, 238, 128,0)', 'rgba(238, 197, 103, 0.5)', '#e39c4e', '#d67034', 
                       '#c83d16', '#9c260c', '#6f1306', '#460201', '#000000']
    
    ba_contour = magics.mcont(
        legend                            = "on",
        contour                           = "off",
        contour_level_selection_type      = "level_list",
        contour_level_list                = [0.,100.,200.,300.,400.,500.,600.,700.,800.],
        contour_gradients_step_list       = [10,10,10,10,10,10,10],
        contour_label                     = "off",
        contour_shade                     = "on",
        contour_shade_colour_method       = "gradients",
        contour_gradients_technique       = "rgb",
        contour_shade_technique           = "cell_shading",
        contour_shade_cell_resolution     = 80,
        contour_gradients_colour_list     = ba_colour_list3,
        contour_gradients_waypoint_method = 'left' )

    ba_line = magics.mcont(
        legend                            = "off",
        contour                           = "on",
        contour_label                     = 'on',
        contour_label_frequency           = 1,
        contour_line_colour               = '#460201',
        contour_highlight                 = "off",
        contour_line_thickness            = 2,
        contour_level_selection_type      = "level_list",
        contour_level_list                = [10.,500],
        contour_shade                     = "off")

    #Defining the legend
    legend = magics.mlegend(
        legend_display_type        = "continuous",
        legend_box_mode            = "automatic",
        legend_title               = "off",
        legend_automatic_position  = "top",
        legend_text_composition    = "user_text_only",
        legend_values_list         = [0.,100.,200.,300.,400.,500.,600.,700.,800.],
        legend_text_font_size      = "0.4",
        legend_text_colour         = "charcoal",    
        legend_entry_border        = "off",
        legend_entry_border_colour = "none") 

    #Setting the title
    date_str1 = ba_time.strftime('%B %Y')
    date_str2 = veg_time.strftime('%B %Y')    
    title = magics.mtext( 
        text_lines          = ['Burned Area Month:  ' +  date_str1 + '  Vegetation Month :  ' +  date_str2, 
                               'Variable in green:  ' + veg_one[parameter].long_name,
                               'Red :  ' + ba_long_name + ' in km<sup>2</sup>, line at 10km<sup>2</sup>'],
        text_justification  = 'left',
        text_font_size      = 0.7,
        text_mode           = "automatic",
        text_colour         = "charcoal") 

    
    clear_output()
    
    return display(magics.plot(projection,background, 
                               veg_data, veg_contour, ba_data, ba_contour, ba_line, 
                               foreground, legend, title))
   

In [8]:

output = widgets.Output(layout={'border': '1px solid black', 'width': '450px'})
style = {'description_width': 'initial', 'width' : '400px'}
layout=Layout(width='500px', height='auto')
largelayout=Layout(width='1000px', height='auto')

style = {'description_width': 'initial'}

drop_p = widgets.Dropdown(options=['abg_avitabile_vod5th', 
                                   'abg_avitabile_vodmean', 
                                   'abg_avitabile_vod95th'],
#                                   'abg_baccini_vod5th',
#                                   'abg_baccini_vodmean', 
#                                   'abg_baccini_vod95th',
#                                   'abg_saatchi_vod5th', 
#                                   'abg_saatchi_vodmean', 
#                                   'abg_saatchi_vod95th',                                  ],
                          value='abg_avitabile_vod5th',
                          description='ABG Parameter',
                          style=style,
                          disabled=False,
                          )

drop_ba = widgets.Dropdown(options=['burned_area', 
                                    'burned_area_in_vegetation_class'],
                          value='burned_area',
                          description='BA Parameter ',
                          style=style,
                          disabled=False,
                          )

veg_class = [i for i in range(18)]

drop_vc = widgets.Dropdown(options=veg_class,
                           value=1,
                           description='Vegetation class',
                           style=style,
                           disabled=False,
                          )

dd = [d.strftime('%Y-%m-%d') for d in pd.date_range('20170101','20171201',freq='MS')]

drop_d = widgets.Dropdown(options=dd,
                          value=dd[0],
                          description='Date',
                          disabled=False
                         )

drop_l = widgets.Dropdown(options=[0,1,3],
                          value=0,
                          description='Lag time',
                          disabled=False
                         )

areas = [('Global','global'), 
         ('Europe','europe'),
         ('North East Europe','north_east_europe'), 
         ('North West Europe','north_west_europe'), 
         ('Central Europe','central_europe'),
         ('South West Europe','south_west_europe'),
         ('South East Europe','south_east_europe'),
         ('East tropics','east_tropic'),  
         ('West tropics','west_tropic'),
         ('Southern Asia','southern_asia'),
         ('South East Asia and Indonesia','south_east_asia_and_indonesia'),
         ('Eastern Asia','eastern_asia'),
         ('Western Asia','western_asia'),
         ('Northern Africa','northern_africa'), 
         ('Southern Africa','southern_africa'),
         ('North Atlantic','north_atlantic'),
         ('Pacific','pacific'), 
         ('Equatorial Pacific','equatorial_pacific'),
         ('South Atlantic and Indian ocean','south_atlantic_and_indian_ocean'),
         ('Arctic','arctic'),
         ('Antarctic','antarctic'),
         ('North pole','north_pole'), 
         ('South pole','south_pole'), 
         ('North America','north_america'),
         ('Central America','central_america'), 
         ('South America','south_america'), 
         ('Middle East and India','middle_east_and_india'), 
         ('Australasia','australasia')]

drop_a = widgets.Dropdown(options=areas,
                          value='northern_africa',
                          description='Area',
                          disabled=False
                         )
try:
    out = widgets.interactive_output(plot, {'parameter'        : drop_p,
                                            'parameter_ba'     : drop_ba,
                                            'vegetation_class' : drop_vc,
                                            'time1'            : drop_d, 
                                            'time_lag'         : drop_l, 
                                            'area'             : drop_a})
except:
    print('Sorry, check that your data is in range Jan 2017 - Dec 2017')
        
widgets.VBox([widgets.HBox([drop_p, drop_d, drop_a]), widgets.HBox([drop_ba, drop_vc, drop_l]), out])